In [1]:
!nvidia-smi

Mon Jan  1 16:06:30 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       Off | 00000000:00:04.0 Off |                    0 |
| N/A   50C    P8              10W /  70W |      0MiB / 15360MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [2]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [3]:
!pip install -q transformers[sentencepiece] ohmeow-blurr fastbook fastai nbdev

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.1/81.1 kB 715.3 kB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 719.8/719.8 kB 7.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.1/66.1 kB 8.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 13.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 887.5/887.5 MB 1.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 507.1/507.1 kB 55.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 6.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.7/58.7 kB 8.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 82.1/82.1 kB 10.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 849.3/849.3 kB 69.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 557.1/557.1 MB 2.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━

In [4]:
import torch
from fastai.text.all import *
from blurr.text.data.all import *
from transformers import AutoConfig
from blurr.text.modeling.all import *
from transformers import AutoModelForSequenceClassification

/usr/local/lib/python3.10/dist-packages/blurr/text/modeling/question_answering.py:31: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  squad_metric = load_metric("squad")
/usr/local/lib/python3.10/dist-packages/datasets/load.py:752: FutureWarning: The repository for squad contains custom code which must be executed to correctly load the metric. You can inspect the repository content at https://raw.githubusercontent.com/huggingface/datasets/2.16.1/metrics/squad/squad.py
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this metric from the next major release of `datasets`.
  warnings.warn(


In [5]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [6]:
%cd /content/drive/MyDrive/keyword_generation/

/content/drive/MyDrive/keyword_generation


In [7]:
df = pd.read_csv("data/papers_final_data.csv")

In [8]:
df.head()

,abstracts,final_keywords
0,"Motivated by the recent explosion of interest around blockchains, we examine whether they make a good fit for the Internet of Things (IoT) sector. Blockchains allow us to have a distributed peer-to-peer network where non-trusting members can interact with each other without a trusted intermediary, in a verifiable manner. We review how this mechanism works and also look into smart contracts-scripts that reside on the blockchain that allow for the automation of multi-step processes. We then move into the IoT domain, and describe how a blockchain-IoT combination: 1) facilitates the sharing of...","['internet of things', 'privacy', 'blockchain']"
1,"At the dawn of the fourth industrial revolution, we are witnessing a fast and widespread adoption of artificial intelligence (AI) in our daily life, which contributes to accelerating the shift towards a more algorithmic society. However, even with such unprecedented advancements, a key impediment to the use of AI-based systems is that they often lack transparency. Indeed, the black-box nature of these systems allows powerful predictions, but it cannot be directly explained. This issue has triggered a new debate on explainable AI (XAI). A research field holds substantial promise for improvi...","['machine learning', 'prediction algorithms', 'biological system modeling']"
2,"The Internet of Things (IoT) makes smart objects the ultimate building blocks in the development of cyber-physical smart pervasive frameworks. The IoT has a variety of application domains, including health care. The IoT revolution is redesigning modern health care with promising technological, economic, and social prospects. This paper surveys advances in IoT-based health care technologies and reviews the state-of-the-art network architectures/platforms, applications, and industrial trends in IoT-based health care solutions. In addition, this paper analyzes distinct IoT security and privac...","['internet of things', 'biological system modeling', 'security']"
3,"In the near future, i.e., beyond 4G, some of the prime objectives or demands that need to be addressed are increased capacity, improved data rate, decreased latency, and better quality of service. To meet these demands, drastic improvements need to be made in cellular network architecture. This paper presents the results of a detailed survey on the fifth generation (5G) cellular network architecture and some of the key emerging technologies that are helpful in improving the architecture and meeting the demands of users. In this detailed survey, the prime focus is on the 5G cellular network...","['5g mobile communication', 'cloud computing']"
4,"The future of mobile communications looks exciting with the potential new use cases and challenging requirements of future 6th generation (6G) and beyond wireless networks. Since the beginning of the modern era of wireless communications, the propagation medium has been perceived as a randomly behaving entity between the transmitter and the receiver, which degrades the quality of the received signal due to the uncontrollable interactions of the transmitted radio waves with the surrounding objects. The recent advent of reconfigurable intelligent surfaces in wireless communications enables, ...",['5g mobile communication']


In [9]:
df.columns

Index(['abstracts', 'final_keywords'], dtype='object')

In [10]:
type(df.iloc[0]['final_keywords'])

str

In [11]:
for index in range(len(df)):
  df.iloc[index]['final_keywords'] = [ item.replace(" '","").replace("'","").lower() for item in re.split(",", df.iloc[index]['final_keywords'].replace("[","").replace("]","")) ]

In [12]:
type(df.iloc[0]['final_keywords'])

list

In [13]:
available_keywords = {}

keywords_list = df['final_keywords'].to_list()

for item in range(len(keywords_list)):
  for keyword in keywords_list[item]:
    if keyword in available_keywords.keys():
      available_keywords[keyword] += 1
    else:
      available_keywords[keyword] = 1

In [14]:
available_keywords

{'internet of things': 770,
 'privacy': 361,
 'blockchain': 392,
 'machine learning': 1496,
 'prediction algorithms': 420,
 'biological system modeling': 448,
 'security': 926,
 '5g mobile communication': 564,
 'cloud computing': 708,
 'wireless communication': 1028,
 'wireless sensor networks': 683,
 'antenna arrays': 362,
 'bandwidth': 894,
 'imaging': 507,
 'remote sensing': 1558,
 'computational modeling': 1449,
 'neural networks': 778,
 'task analysis': 2087,
 'predictive models': 1147,
 'deep learning': 1954,
 'training': 2263,
 'testing': 391,
 'support vector machines': 755,
 'databases': 370,
 'artificial intelligence': 372,
 'logic gates': 974,
 'servers': 513,
 'computer architecture': 734,
 'feature extraction': 3802,
 'optimization': 2152,
 'data mining': 756,
 'real-time systems': 794,
 'convergence': 411,
 'convolution': 663,
 'convolutional neural networks': 714,
 'optical fiber communication': 461,
 'radio frequency': 699,
 'modulation': 642,
 'algorithm design and ana

In [15]:
encode_keywords = { key: idx for idx, (key, value) in enumerate(available_keywords.items())}
with open("encoded_keywords.json", "w") as fp:
  json.dump(encode_keywords, fp)

In [16]:
encode_keywords

{'internet of things': 0,
 'privacy': 1,
 'blockchain': 2,
 'machine learning': 3,
 'prediction algorithms': 4,
 'biological system modeling': 5,
 'security': 6,
 '5g mobile communication': 7,
 'cloud computing': 8,
 'wireless communication': 9,
 'wireless sensor networks': 10,
 'antenna arrays': 11,
 'bandwidth': 12,
 'imaging': 13,
 'remote sensing': 14,
 'computational modeling': 15,
 'neural networks': 16,
 'task analysis': 17,
 'predictive models': 18,
 'deep learning': 19,
 'training': 20,
 'testing': 21,
 'support vector machines': 22,
 'databases': 23,
 'artificial intelligence': 24,
 'logic gates': 25,
 'servers': 26,
 'computer architecture': 27,
 'feature extraction': 28,
 'optimization': 29,
 'data mining': 30,
 'real-time systems': 31,
 'convergence': 32,
 'convolution': 33,
 'convolutional neural networks': 34,
 'optical fiber communication': 35,
 'radio frequency': 36,
 'modulation': 37,
 'algorithm design and analysis': 38,
 'data models': 39,
 'monitoring': 40,
 'time 

In [17]:
categorical_keywords_list = []

for keywords in keywords_list:
  categorical_list = [0]*len(encode_keywords)
  for keyword in keywords:
    keyword_index = encode_keywords[keyword]
    categorical_list[keyword_index] = 1
  categorical_keywords_list.append(categorical_list)

In [18]:
df['encoded_keywords'] = categorical_keywords_list

In [19]:
df.shape

(36398, 3)

In [20]:
df.tail()

,abstracts,final_keywords,encoded_keywords
36393,"Wafer level packaging could reduce the cost of MEMS based sensors through simplified processing and inexpensive components. A novel vacuum tight seal referred to as the wedge seal method is proposed in this paper. The seal consists of a silicon wedge forced into a pliable material (typically a metal) that is attached to the component wafer. The wedge-seal addresses some of the requirements of micro-bolometer packaging in that it provides a vacuum tight seal at low temperatures with tolerance to surface finish and topography. The concept was evaluated with regards to manufacturability, mate...","[silicon, sensors]","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...]"
36394,"In practical image storage and distribution systems. JPEG is a common image format. In this paper, we propose a blind watermarking method for JPEG-compressed color images in the semi-decompression domain by integrating the JPEG coding process. The watermark is embedded by modifying the quantized DC coefficients based on the spread spectrum technique. There are some modifications on the basic spread spectrum watermarking scheme to meet the sensitivity of the quantized DC coefficients. Furthermore, Reed-Solomon coding is employed to improve the robustness of watermarking. Experimental result...","[visualization, sensitivity, modulation]","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...]"
36395,"This paper reports the effects of the following substrates on the optical, structural and morphological properties of ZnO thin films: Si (100), Si (111), c- and r-sapphire, glass, GaAs and ZnO. The thin films were deposited by low pressure Metal Organic Chemical Vapor Deposition using diethylzinc and tertiary butanol as the zinc and oxygen sources, respectively. X-ray diffraction analysis shows that all the ZnO layers are c-axis orientated. Scanning Electron microscopy reveals similar morphology for all the substrates used, with hexagonal columns having cone shape ends being evident. The p...","[substrates, silicon]","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...]"
36396,"The stability margin of a two degree-of-freedom self-sensing active magnetic bearing (AMB) is estimated by means of μ-analysis. The specific self-sensing algorithm implemented in this study is the direct current measurement method. Detailed black-box models are developed for the main subsystems in the AMB by means of discrete-time system identification. In order to obtain models for dynamic uncertainty in the various subsystems in the AMB, the identified models are combined to form a closed-loop model for the self-sensing AMB. The response of this closed-loop model is then compared to the ...","[uncertainty, integrated circuit modeling, analytical models, robustness, rotors]","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...]"
36397,How learning is conceptualized and negotiated within the engineering undergraduate curriculum is affected by the th

In [21]:
labels = list(encode_keywords.keys())

In [22]:
labels

['internet of things',
 'privacy',
 'blockchain',
 'machine learning',
 'prediction algorithms',
 'biological system modeling',
 'security',
 '5g mobile communication',
 'cloud computing',
 'wireless communication',
 'wireless sensor networks',
 'antenna arrays',
 'bandwidth',
 'imaging',
 'remote sensing',
 'computational modeling',
 'neural networks',
 'task analysis',
 'predictive models',
 'deep learning',
 'training',
 'testing',
 'support vector machines',
 'databases',
 'artificial intelligence',
 'logic gates',
 'servers',
 'computer architecture',
 'feature extraction',
 'optimization',
 'data mining',
 'real-time systems',
 'convergence',
 'convolution',
 'convolutional neural networks',
 'optical fiber communication',
 'radio frequency',
 'modulation',
 'algorithm design and analysis',
 'data models',
 'monitoring',
 'time series analysis',
 'convolutional neural network',
 'classification algorithms',
 'object detection',
 'detectors',
 'strain',
 'standards',
 'solid model

In [23]:
model_name = "xlm-roberta-large"

model_cls = AutoModelForSequenceClassification

config = AutoConfig.from_pretrained(model_name)

config.num_labels = len(labels)

hf_arch, hf_config, hf_tokenizer, hf_model = get_hf_objects(model_name, model_cls = model_cls, config = config)

hf_model.config.problem_type = "multi_label_classification"

config.json:   0%|          | 0.00/616 [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.10M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/2.24G [00:00<?, ?B/s]

In [25]:
blocks = (TextBlock(hf_arch, hf_config, hf_tokenizer, hf_model), MultiCategoryBlock(encoded=True, vocab=labels))

dblock = DataBlock(
    blocks=blocks,
    get_x = ColReader('abstracts'),
    get_y = ColReader('encoded_keywords'),
    splitter = RandomSplitter(valid_pct = 0.1, seed = 42)
)


In [26]:
batch_size = 128

In [27]:
dls = dblock.dataloaders(df, bs=batch_size)

In [28]:
dataloaders_path = "dataloaders"

In [30]:
torch.save(dls, f"{dataloaders_path}/dls_xlm_roberta_0.003.pkl")

In [31]:
dls.show_batch(dataloaders = dls, max_n = 4, trunc_at = 512)

,text,target
0,"In a mining district such as ours, where a large proportion of the engineers are either directly or indirectly connected with the mines, it is surprising to note how little the subject of signalling in the mines has been dealt with in technical papers. Perhaps it is because the importance of the signalling system is not realised to the extent it deserves. The development of railways would have been considerably retarded without the telegraph, and I need add no words to convince you that the same would apply",[resistance]
1,"In the past few years, a large body of literature has been created on downlink Non-Orthogonal Multiple Access (NOMA), employing superposition coding and Successive Interference Cancellation (SIC), in multi-antenna wireless networks. Furthermore, the benefits of NOMA over Orthogonal Multiple Access (OMA) have been highlighted. In this paper, we take a critical and fresh look at the downlink Next Generation Multiple Access (NGMA) literature. Instead of contrasting NOMA with OMA, we contrast NOMA with two othe","[receivers, complexity theory]"
2,"COVID-19 is a newly identified disease, which is very contagious and has been rapidly spreading across different countries around the world, calling for rapid and accurate diagnosis tools. Chest CT imaging has been widely used in clinical practice for disease diagnosis, but image reading is still a time-consuming work. We aim to integrate an image preprocessing technology for anomaly detection with supervised deep learning for chest CT imaging-based COVID-19 diagnosis. In this study, a matrix profile techni","[deep learning, feature extraction, image segmentation]"
3,"Estimating gross primary production (GPP), the gross uptake of CO 2 by vegetation, is a fundamental prerequisite for understanding and quantifying the terrestrial carbon cycle. Over the last decade, multiple approaches have been developed to derive spatiotemporal dynamics of GPP combining in situ observations and remote sensing data using machine learning techniques or semiempirical models. However, no high spatial resolution GPP product exists so far that is derived entirely from satellite-based remote sen","[machine learning, biological system modeling, remote sensing, indexes, spatial resolution]"
